# LiteLLM Proxy Connection Diagnosis

This notebook diagnoses connection issues to the LiteLLM Proxy.

It performs multiple steps to isolate why the API Key might be missing:
1. **Direct HTTP Request**: Verifies network, API Key, and Endpoint validity using `requests`.
2. **LiteLLM SDK Test (Standard)**: Standard `litellm.completion` call.
3. **LiteLLM SDK Test (Manually Constructed URL)**: Investigating exact URL endpoint.

In [ ]:
import os
import requests
from litellm import completion
from dotenv import load_dotenv

# 1. Load Environment Variables
env_path = os.path.join(os.path.dirname(os.getcwd()), 'backend', '.env')
print(f"Loading .env from: {env_path}")
load_dotenv(env_path)

API_KEY = os.environ("LITELLM_PROXY_API_KEY")
API_BASE = "http://sacahan-ubunto:4000"
MODEL = "gpt-4o"

print(f"PROXY URL: {API_BASE}")
print(f"MODEL: {MODEL}")
print(f"API KEY: {API_KEY[:4]}...{API_KEY[-4:] if API_KEY else 'None'}")

## Step 1: Direct connectivity test
This tests if the server is reachable and valid at the `/chat/completions` endpoint.
**Note:** We found the proxy listens at `/chat/completions`, NOT `/v1/chat/completions`.

In [ ]:
# Ensure base doesn't have trailing slash
if API_BASE.endswith('/'):
    base = API_BASE[:-1]
else:
    base = API_BASE

# Construct explicit URL
url = f"{base}/chat/completions"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

data = {
    "model": MODEL,
    "messages": [{"role": "user", "content": "Hello, direct test!"}]
}

print(f"Sending Direct Request to: {url}")
try:
    resp = requests.post(url, json=data, headers=headers, timeout=10)
    print(f"Status Code: {resp.status_code}")
    if resp.status_code == 200:
        print("✅ Direct Connection Successful")
        print("Response:", resp.json()['choices'][0]['message']['content'])
    else:
        print("❌ Direct Connection Failed")
        print("Body:", resp.text)
except Exception as e:
    print(f"❌ Exception during request: {e}")

## Step 2: OpenAI SDK Test


In [ ]:
# Enable verbose logging
from openai import OpenAI

client = OpenAI(
    base_url=API_BASE,  # 基礎 URL，不含 /chat/completions
    api_key=API_KEY,
)

try:
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"content": "Hello from LiteLLM!", "role": "user"}],
    )
    print("\n✅ OpenAI Success!")
    print(response.choices[0].message.content)
except Exception as e:
    print("\n❌ OpenAI Failed:")
    print(e)

## Step 3: LiteLLM SDK Test (Corrected URL)
If the previous test failed with 'Request blocked', it implies incorrect endpoint construction (e.g., trying to hit root).
We will try to force the path to `/chat/completions` by appending it to `api_base` IF we think LiteLLM strips it, OR verify path mechanics.

In [ ]:
# Enable verbose logging
import litellm
litellm._turn_on_debug()

print("\n--- Experiment 5: Manually append /chat/completions to api_base ---")
# Normally 'openai' provider appends /chat/completions. But if API_BASE is treated as the full URL, we might need to specify exact path.
# However, if we append it, does LiteLLM append it again? Let's check logs.
manual_url = f"{API_BASE}/chat/completions"
print(f"Testing api_base='{manual_url}'")
try:
    response = completion(
        model=MODEL,
        messages=[{"content": "Hello from LiteLLM (Manual URL)!", "role": "user"}],
        base_url=API_BASE,
        api_key=API_KEY,
        # custom_llm_provider="openai",
        # extra_headers={
        #     "Content-Type": "application/json",
        #     "Authorization": f"Bearer {API_KEY}",
        # },
    )
    print("✅ Experiment 5 Success!")
    print(response.choices[0].message.content)
except Exception as e:
    print("❌ Experiment 5 Failed:")
    print(e)